In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import syft as sy

In [2]:
# 创建工作机
hook = sy.TorchHook(torch)
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [13]:
# 模型定义
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)
    def forward(self, x):
        x = x.reshape(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

model = FCNN()
model

FCNN(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)

In [14]:
# 创建数据
X = torch.rand(784, 784).fix_precision(precision_fractional=4)
X.shape

torch.Size([784, 784])

In [15]:
# 将数据发送给alice和bob
X_ = X.share(bob, alice, crypto_provider=crypto_provider)
X_


(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:83595412702 -> bob:90977950707]
	-> [PointerTensor | me:56593571348 -> alice:52839573047]
	*crypto provider: crypto_provider*

In [16]:
# 前向传播
Y_ = model(X_)
Y = Y_.get()
Y

RuntimeError: matrices expected, got 1D, 2D tensors at /pytorch/aten/src/TH/generic/THTensorMath.cpp:131